In [1]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "info": "#76FF7B",
        "warning": "#FBDDFE",
        "error": "#FF0000",
    }
)
console = Console(theme=custom_theme)

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Add seed
np.random.seed(0)

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from utils import get_data_summary, get_value_counts, Preparedata

### Load Data

In [3]:
fp: str = "../data/titanic_data.csv"

data_original: pl.DataFrame = pl.read_csv(source=fp)
data_original.head()

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
i64,i64,str,str,f64,i64,i64,str,f64,str,str,str,i64,str
1,1,"""Allen, Miss. E…","""female""",29.0,0,0,"""24160""",211.3375,"""B5""","""S""","""2""",null,"""St Louis, MO"""
1,1,"""Allison, Maste…","""male""",0.9167,1,2,"""113781""",151.55,"""C22""","""S""","""11""",null,"""Montreal, PQ /…"
1,0,"""Allison, Miss.…","""female""",2.0,1,2,"""113781""",151.55,"""C22""","""S""",null,null,"""Montreal, PQ /…"
1,0,"""Allison, Mr. H…","""male""",30.0,1,2,"""113781""",151.55,"""C22""","""S""",null,135,"""Montreal, PQ /…"
1,0,"""Allison, Mrs. …","""female""",25.0,1,2,"""113781""",151.55,"""C22""","""S""",null,null,"""Montreal, PQ /…"


In [4]:
# Config
TARGET: str = "survived"
TEST_SIZE: float = 0.2
RANDOM_STATE: int = 123

In [5]:
get_data_summary(data=data_original, features=data_original.columns)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
count,1309.000000,1309.000000,1309,1309,1046.000000,1309.000000,1309.000000,1309,1308.000000,295,1307,486,121.000000,745
unique,NaN,NaN,1307,2,NaN,NaN,NaN,929,NaN,181,3,27,NaN,369
top,NaN,NaN,"Connolly, Miss. Kate",male,NaN,NaN,NaN,CA. 2343,NaN,F,S,13,NaN,"New York, NY"
freq,NaN,NaN,2,843,NaN,NaN,NaN,11,NaN,8,914,39,NaN,64
mean,2.294882,0.381971,NaN,NaN,29.881135,0.498854,0.385027,NaN,33.295479,NaN,NaN,NaN,160.809917,NaN
std,0.837836,0.486055,NaN,NaN,14.413500,1.041658,0.865560,NaN,51.758668,NaN,NaN,NaN,97.696922,NaN
min,1.000000,0.000000,NaN,NaN,0.166700,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,1.000000,NaN
25%,2.000000,0.000000,NaN,NaN,21.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN,NaN,72.000000,NaN
50%,3.000000,0.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN,NaN,155.000000,NaN
75%,3.000000,1.000000,NaN,NaN,39.000000,1.000000,0.000000,NaN,31.275000,NaN,NaN,NaN,256.000000,NaN


In [6]:
# Drop columns with too many NaNs or unnecessary columns
cols_to_drop: list[str] = ["cabin", "boat", "body", "home.dest"]
data: pl.DataFrame = data_original.drop(columns=cols_to_drop)

get_data_summary(data=data, features=data.columns)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked
count,1309.000000,1309.000000,1309,1309,1046.000000,1309.000000,1309.000000,1309,1308.000000,1307
unique,NaN,NaN,1307,2,NaN,NaN,NaN,929,NaN,3
top,NaN,NaN,"Connolly, Miss. Kate",male,NaN,NaN,NaN,CA. 2343,NaN,S
freq,NaN,NaN,2,843,NaN,NaN,NaN,11,NaN,914
mean,2.294882,0.381971,NaN,NaN,29.881135,0.498854,0.385027,NaN,33.295479,NaN
std,0.837836,0.486055,NaN,NaN,14.413500,1.041658,0.865560,NaN,51.758668,NaN
min,1.000000,0.000000,NaN,NaN,0.166700,0.000000,0.000000,NaN,0.000000,NaN
25%,2.000000,0.000000,NaN,NaN,21.000000,0.000000,0.000000,NaN,7.895800,NaN
50%,3.000000,0.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN
75%,3.000000,1.000000,NaN,NaN,39.000000,1.000000,0.000000,NaN,31.275000,NaN


In [7]:
data.head(10)

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked
i64,i64,str,str,f64,i64,i64,str,f64,str
1,1,"""Allen, Miss. E…","""female""",29.0,0,0,"""24160""",211.3375,"""S"""
1,1,"""Allison, Maste…","""male""",0.9167,1,2,"""113781""",151.55,"""S"""
1,0,"""Allison, Miss.…","""female""",2.0,1,2,"""113781""",151.55,"""S"""
1,0,"""Allison, Mr. H…","""male""",30.0,1,2,"""113781""",151.55,"""S"""
1,0,"""Allison, Mrs. …","""female""",25.0,1,2,"""113781""",151.55,"""S"""
1,1,"""Anderson, Mr. …","""male""",48.0,0,0,"""19952""",26.55,"""S"""
1,1,"""Andrews, Miss.…","""female""",63.0,1,0,"""13502""",77.9583,"""S"""
1,0,"""Andrews, Mr. T…","""male""",39.0,0,0,"""112050""",0.0,"""S"""
1,1,"""Appleton, Mrs.…","""female""",53.0,2,0,"""11769""",51.4792,"""S"""


In [10]:
data_with_index = data.with_row_index("id", offset=0)
data_with_index

id,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked
u32,i64,i64,str,str,f64,i64,i64,str,f64,str
0,1,1,"""Allen, Miss. E…","""female""",29.0,0,0,"""24160""",211.3375,"""S"""
1,1,1,"""Allison, Maste…","""male""",0.9167,1,2,"""113781""",151.55,"""S"""
2,1,0,"""Allison, Miss.…","""female""",2.0,1,2,"""113781""",151.55,"""S"""
3,1,0,"""Allison, Mr. H…","""male""",30.0,1,2,"""113781""",151.55,"""S"""
4,1,0,"""Allison, Mrs. …","""female""",25.0,1,2,"""113781""",151.55,"""S"""
…,…,…,…,…,…,…,…,…,…,…
1304,3,0,"""Zabour, Miss. …","""female""",14.5,1,0,"""2665""",14.4542,"""C"""
1305,3,0,"""Zabour, Miss. …","""female""",null,1,0,"""2665""",14.4542,"""C"""
1306,3,0,"""Zakarian, Mr. …","""male""",26.5,0,0,"""2656""",7.225,"""C"""


In [ ]:
# Sex: {"female": 0, "male": 1}, Embarked: {"S": 0, "C": 1, "Q": 2},

data.lazy().with_columns(
    sex_int=pl.col("sex").cast(pl.Categorical).to_physical(),
    embarked_int=pl.col("embarked").cast(pl.Categorical).to_physical(),
    ticket_int=pl.col("ticket").str.contains(r"^\D"),
).with_columns(
    ticket_int=(
        pl.col("ticket_int")
        .cast(pl.Categorical)
        .to_physical()  # similar to pd.factorize
    )
).collect()

In [ ]:
# Split the data
from sklearn.model_selection import train_test_split

X_train: pl.DataFrame
X_test: pl.DataFrame

X_train, X_test = train_test_split(
    data_original,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=data_original.select(TARGET),
)

X_train.shape, X_test.shape

In [ ]:
X_train.head()

In [ ]:
# The name is the U_id (Polars does NOT maintain order)
y_train: pd.DataFrame = (
    X_train.select(["name", TARGET]).sort("name").drop(columns=["name"]).to_pandas()
)
y_test: pd.DataFrame = (
    X_test.select(["name", TARGET]).sort("name").drop(columns=["name"]).to_pandas()
)

y_train.head()

In [ ]:
console.print(f"Train: {get_value_counts(data=y_train, feature=TARGET)}")
console.print(f"Test: {get_value_counts(data=y_test, feature=TARGET)}")

In [ ]:
num_vars: list[str] = ["pclass", "age", "sibsp", "parch", "ticket", "fare"]
cat_vars: list[str] = ["sex", "embarked"]

get_data_summary(data=X_train, features=X_train.columns)

In [ ]:
processor = Preparedata(variables=cat_vars + ["ticket"])
processor.fit_transform(X=X_train)

In [ ]:
from feature_engine.selection import DropFeatures
from feature_engine.imputation import CategoricalImputer, MeanMedianImputer
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


set_config(transform_output="pandas")

col_transf: ColumnTransformer = ColumnTransformer(
    transformers=[
        ("num_vars", MinMaxScaler(clip=True), num_vars),
        (
            "cat_vars",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            cat_vars,
        ),
    ],
    remainder="drop",
)
preprocessor: Pipeline = Pipeline(
    steps=[
        ("drop_features", DropFeatures(features_to_drop=cols_to_drop)),
        ("preprocess", Preparedata(variables=["sex", "embarked", "ticket"])),
        ("median_imputer", MeanMedianImputer(variables=num_vars)),
        (
            "cat_imputer",
            CategoricalImputer(
                imputation_method="frequent", variables=cat_vars, ignore_format=True
            ),
        ),
        ("col_transf", col_transf),
    ]
)
preprocessor

In [ ]:
X_train_tr: pd.DataFrame = preprocessor.fit_transform(X=X_train.to_pandas())
X_test_tr: pd.DataFrame = preprocessor.transform(X=X_test.to_pandas())

X_train_tr.head()

In [ ]:
assert X_train_tr.shape[0] == y_train.shape[0], "The size is NOT equal!"
assert X_test_tr.shape[0] == y_test.shape[0], "The size is NOT equal!"

In [ ]:
from mlflow_example import Experiment, Estimator, TrainingData, run_experiment
import mlflow
from sklearn.linear_model import LogisticRegression

In [ ]:
training_data: TrainingData = TrainingData(
    X_train=X_train_tr,
    X_validate=X_test_tr,
    y_train=y_train[TARGET],
    y_validate=y_test[TARGET],
)

experiment: Experiment = Experiment(
    experiment_name="test experiment",
    experiment_type="classification",
    run_name="test",
    model_name="log-model2",
    # tracking_uri="http://127.0.0.1:5252", # I don't know why it doesn't work on docker.
    tracking_uri="http://localhost:5252",
)

log_model: LogisticRegression = LogisticRegression(random_state=123)
estimator: Estimator = Estimator(
    preprocessor=preprocessor,
    model=log_model,
)

### Start The MLFlow Traking Server

- Using `Localhost` and `SQLite`

```sh
mlflow server -h "127.0.0.1" -p 5252 --backend-store-uri sqlite:///mlruns.db
```

In [ ]:
run_experiment(
    experiment=experiment,
    estimator=estimator,
    training_data=training_data,
)

In [ ]:
from typing import Literal
from mlflow import MlflowClient

client = MlflowClient()
validation_tag: Literal["pending", "passed"]
validation_tag = "passed"
model_version: int = 1

# Add tag
client.set_registered_model_tag(
    experiment.model_name, "model.validation_status", validation_tag
)
client.set_registered_model_tag(experiment.model_name, "model.data_scientist", "Neidu")

In [ ]:
cd / && . venv/bin/activate \
    && cd app/src \
    && python test.py